In [ ]:
""" To Do List
1)Find suitable groups based on spining data
2)Fins theh x,y,z distribution for all of the neurites
Find the length distribution for all of the neurites
Find the volume distribution for all of the neurites
Find the x, y, z distribution for all of the spining groups
For For all spining groups:
Take out the dendrite numbers and filter by size
Do heat map of compartments


"""

In [2]:
import datajoint as dj
import numpy as np
import time

from meshparty import trimesh_io
import pandas as pd
import seaborn as sns
sns.set() #sets the asethetics parameters in what set

ModuleNotFoundError: No module named 'meshparty'

In [ ]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


In [ ]:
#ta3p100.NeuriteTargetsRevised.describe()

In [ ]:
#NeuriteTargets was recomputed from the NeuriteRevised list
len(ta3p100.NeuriteTargetsRevised() & "spine_num<total"),len(ta3p100.NeuriteTargetsRevised() )

In [ ]:
ta3p100.NeuriteTargetsRevised()

In [ ]:
#creates the table with spine data without the spine label
#how to filter out the spine proportion

#have to first remove the ones that have only spine synapses


neurite_targets_no_spine = (ta3p100.NeuriteTargetsRevised() & "(spine_num + spine_neck_num) <total").proj(
                                                         spine_head_prop="spine_head_prop/(1-spine_prop - spine_neck_prop)",
                                                         shaft_prop="shaft_prop/(1-spine_prop - spine_neck_prop)",
                                                         #non_spine_prop="non_spine_prop/(1-spine_prop - spine_neck_prop)",
                                                         soma_prop = "soma_prop/(1-spine_prop - spine_neck_prop)",
                                                         axon_prop = "axon_prop/(1-spine_prop - spine_neck_prop)",
                                                         total="total-spine_num - spine_neck_num",
                                            )

neurite_targets_no_spine



In [ ]:
#different lengths based on the number of synapses threshold

print(len(neurite_targets_no_spine))
print(len(neurite_targets_no_spine & "total>12"))
print(len(neurite_targets_no_spine & "total>10")) 
print(len(neurite_targets_no_spine & "total>8")) 
print(len(neurite_targets_no_spine & "total>6")) 

# Manually seperating the groups into: Axon, Soma, Head, Shaft

In [ ]:
neurite_targets_no_spine_6 = (neurite_targets_no_spine & "total>6")
print("Total in neurite table with greater than 6 synapses = " + str(len(neurite_targets_no_spine_6)))

In [ ]:
#show the distribution
axon_prop = neurite_targets_no_spine_6.fetch("axon_prop")
axon_prop
axon_greater_0_array = axon_prop[axon_prop.astype("float") > 0]
sns.distplot(axon_greater_0_array.astype("float"))

In [ ]:
#show the distribution
soma_prop = (neurite_targets_no_spine_6 ).fetch("soma_prop")
soma_prop
soma_greater_0_array = soma_prop[soma_prop.astype("float") > 0]
sns.distplot(soma_greater_0_array.astype("float"))

In [ ]:
#manually find the groups
axon_group = neurite_targets_no_spine_6 & "axon_prop>0"
print("Axon group = " + str(len(axon_group)))
soma_group = (neurite_targets_no_spine_6 - axon_group.proj()) & "soma_prop>=0.25"
print("Soma group = " + str(len(soma_group)))

In [ ]:
#remove the axon and soma groups
neurite_targets_no_spine_6_without_axon_soma = neurite_targets_no_spine_6 - axon_group.proj() - soma_group.proj()
len(neurite_targets_no_spine_6_without_axon_soma)/len(neurite_targets_no_spine_6)

In [ ]:
#get heatmap of the data

neurite_spine_data = neurite_targets_no_spine_6_without_axon_soma.fetch(as_dict=True)

spine_df = pd.DataFrame(neurite_spine_data)
spine_df_base = spine_df.drop(columns=['segmentation', 'segment_id',"total","axon_prop","soma_prop"])
#spine_df_base

In [ ]:
#sort by the first column and then do heat map
spine_df_base_head_shaft_sort = spine_df_base.sort_values(['spine_head_prop','shaft_prop'], ascending=[False,True])
spine_df_base_head_shaft_sort

In [ ]:
#heat map of spine heads
spine_head_shaft_sort = sns.heatmap(spine_df_base_head_shaft_sort)

In [ ]:
#making the head group
head_group = neurite_targets_no_spine_6_without_axon_soma & "spine_head_prop >=0.6"
len(head_group)

In [ ]:
#heat map of spine heads
spine_shaft = spine_df_base_head_shaft_sort[spine_df_base_head_shaft_sort["spine_head_prop"]<0.6].sort_values("shaft_prop",ascending=True)
#spine_df_base_head_shaft_sort = spine_df_base.sort_values(['spine_head_prop','shaft_prop'], ascending=[False,True])
spine_head_shaft_sort = sns.heatmap(spine_shaft)

In [ ]:
#making the shaft group
shaft_group = (neurite_targets_no_spine_6_without_axon_soma & "shaft_prop >=0.5") - head_group.proj()
len(shaft_group)

In [ ]:
unused = neurite_targets_no_spine_6_without_axon_soma - head_group.proj() - shaft_group.proj()

In [ ]:
print(len(soma_group),len(axon_group),len(head_group),len(shaft_group),len(unused))
print(sum([len(soma_group),len(axon_group),len(head_group),len(shaft_group),len(unused)]))

In [ ]:
print(len(neurite_targets_no_spine_6))

In [ ]:
ta3p100.LabelKey.describe()

In [ ]:
ta3p100.SpineClustersKey.drop()

In [ ]:
#look up datajoint table that has the clusters
@schema
class SpineClustersKey(dj.Lookup):
    definition="""
    cluster_id : tinyint unsigned #numeric label of cluster
    ---
    description : varchar(16) #descriptive name of cluster
    threshold   : float #the proportion cut off for the group based on the category
    """
    

In [ ]:
cluster_descriptions = ["Soma","Axon","Spine Head","Shaft","Unused"]
cluster_thresholds = [0.25,0.0001,0.6,0.5,0]

clusters_dict = [dict(cluster_id=i,description=cd,threshold=ct) for i,(cd,ct) in enumerate(zip(cluster_descriptions,cluster_thresholds))]
clusters_dict

In [ ]:
SpineClustersKey.insert(clusters_dict,skip_duplicates=True)

In [ ]:
SpineClustersKey()

In [ ]:
## Creates the table that holds all of the segment groups
@schema
class SpineClusters(dj.Manual):
    definition="""
    -> ta3p100.NeuriteRevised
    ---
    ->SpineClustersKey
    """

In [ ]:
soma_group.proj(cluster_id="0")

In [ ]:
dict = SpineClustersKey.fetch(as_dict=True)
dict

In [ ]:
possible_groups = [soma_group,axon_group,head_group,shaft_group,unused]
descriptors = [0,1,2,3,4]
for i,table in enumerate(possible_groups):
    new_table = table.proj(cluster_id=str(descriptors[i]))
    SpineClusters.insert(new_table,skip_duplicates=True)
    

In [ ]:
(len(SpineClusters() & "cluster_id=0"),len(SpineClusters() & "cluster_id=1"),
 len(SpineClusters() & "cluster_id=2"),len(SpineClusters() & "cluster_id=3"),
 len(SpineClusters() & "cluster_id=4"))

In [ ]:
print(len(soma_group),len(axon_group),len(head_group),len(shaft_group),len(unused))
print(sum([len(soma_group),len(axon_group),len(head_group),len(shaft_group),len(unused)]))